In [1]:
import requests

def retrieve_data(url):
    """
    Retrieve data from a URL.
    """
    try:
        response = requests.get(url, headers={'Content-Type': 'application/json'})
        if response.ok:
            return response
    except:
        print("Error: RemoteNumberDataSource.get()")
        return None

In [2]:
import json

response = retrieve_data("https://bet.hkjc.com/contentserver/jcbw/cmc/last30draw.json")
json_data = json.loads(response.content)
print(json_data[0])

{'id': '22/040', 'date': '27/06/2022', 'no': '1+13+19+26+29+37', 'sno': '15', 'sbcode': 'MOS', 'sbnameE': 'HONG KONG REUNIFICATION SNOWBALL', 'sbnameC': '回歸金多寶', 'inv': '197,454,377', 'p1': '58,730,450', 'p1u': '1.5', 'p2': '2,081,160', 'p2u': '4.5', 'p3': '83,520', 'p3u': '299.0', 'p4': '9,600', 'p4u': '712.0', 'p5': '640', 'p5u': '17,587.5', 'p6': '320', 'p6u': '20,707.1', 'p7': '40', 'p7u': '332,453.7'}


In [3]:
numbers_data = [i["no"] for i in json_data]
numbers_data = [x.split("+") for x in numbers_data]
print(numbers_data[:2])

[['1', '13', '19', '26', '29', '37'], ['2', '15', '20', '30', '47', '48']]


In [4]:
import collections
import numpy as np

def counter_numbers(numbers):
    """
    Count the number of times each number appears in the last 30 draws.
    """
    numbers = np.array(numbers)
    counter = collections.Counter(numbers.flatten())
    return counter

counter = counter_numbers(numbers_data)
print(counter)

Counter({'30': 8, '48': 7, '46': 7, '8': 6, '40': 6, '12': 6, '1': 5, '29': 5, '37': 5, '15': 5, '39': 5, '21': 5, '34': 5, '38': 5, '11': 5, '35': 5, '28': 5, '26': 4, '20': 4, '47': 4, '9': 4, '17': 4, '7': 4, '19': 3, '2': 3, '27': 3, '16': 3, '41': 3, '32': 3, '42': 3, '49': 3, '36': 3, '10': 3, '22': 3, '18': 3, '13': 2, '24': 2, '33': 2, '44': 2, '43': 2, '25': 2, '5': 2, '45': 2, '4': 2, '3': 2, '23': 2, '14': 1, '31': 1, '6': 1})


In [5]:
def retrieve_most_number(counter):
    """
    Get all the most number.
    """
    numbers = []
    most_numbers = {}
    for k in counter.keys():
        index = counter[k]
        numbers = most_numbers[index]  if index in most_numbers else []
        numbers.append(k)
        most_numbers[index] = numbers

    most_numbers = sorted(most_numbers.items(), key=lambda x: x[0], reverse=True)
    return most_numbers

def show_most_number(counter):
    """
    Show the most number.
    """
    print(f'===============Most of numbers==============')
    most_numbers = retrieve_most_number(counter)
    for v in most_numbers:
        key = v[0]
        values = sorted(v[1], key=lambda x: int(x))
        print(f'p{key}', values)

def show_particular_number(counter, by, to):
    """
    Show the particular number.
    """
    print(f'===============[{by},{to}]==============')
    numbers = []
    most_numbers = retrieve_most_number(counter)
    for v in most_numbers:
        numbers.clear()
        key = v[0]
        values = sorted(v[1], key=lambda x: int(x))
        for i in values:
            if int(i) in range(by, to):
                numbers.append(i)
        if len(numbers) > 0: print(f'p{key}', numbers)
        
print(numbers_data[0])
show_most_number(counter)
show_particular_number(counter, 1, 10)
show_particular_number(counter, 10, 20)
show_particular_number(counter, 20, 30)
show_particular_number(counter, 30, 40)
show_particular_number(counter, 40, 50)

['1', '13', '19', '26', '29', '37']
===============Most of numbers==============
p8 ['30']
p7 ['46', '48']
p6 ['8', '12', '40']
p5 ['1', '11', '15', '21', '28', '29', '34', '35', '37', '38', '39']
p4 ['7', '9', '17', '20', '26', '47']
p3 ['2', '10', '16', '18', '19', '22', '27', '32', '36', '41', '42', '49']
p2 ['3', '4', '5', '13', '23', '24', '25', '33', '43', '44', '45']
p1 ['6', '14', '31']
===============[1,10]==============
p6 ['8']
p5 ['1']
p4 ['7', '9']
p3 ['2']
p2 ['3', '4', '5']
p1 ['6']
===============[10,20]==============
p6 ['12']
p5 ['11', '15']
p4 ['17']
p3 ['10', '16', '18', '19']
p2 ['13']
p1 ['14']
===============[20,30]==============
p5 ['21', '28', '29']
p4 ['20', '26']
p3 ['22', '27']
p2 ['23', '24', '25']
===============[30,40]==============
p8 ['30']
p5 ['34', '35', '37', '38', '39']
p3 ['32', '36']
p2 ['33']
p1 ['31']
===============[40,50]==============
p7 ['46', '48']
p6 ['40']
p4 ['47']
p3 ['41', '42', '49']
p2 ['43', '44', '45']


In [6]:
import math

def search_numbers(numbers, by, to):
    """
    Search for a number in a list of numbers.
    """
    slots = []
    for i in numbers:
        for k in i:
            if int(k)in range(by, to):
                slots.append(k)
    return slots

def show_percentage(numbers, by, to):
    """
    Show the percentage of numbers in a list of numbers.
    """
    slots = search_numbers(numbers, by, to)
    percent = math.ceil((len(slots) / (len(numbers) * 6)) * 100)
    print("{} ~ {} : {}%".format(by, to, percent))

show_percentage(numbers_data, 1, 10)
show_percentage(numbers_data, 10, 20)
show_percentage(numbers_data, 20, 30)
show_percentage(numbers_data, 30, 40)
show_percentage(numbers_data, 40, 50)

1 ~ 10 : 17%
10 ~ 20 : 20%
20 ~ 30 : 20%
30 ~ 40 : 24%
40 ~ 50 : 22%


In [10]:
def retrieve_priority_numbers(counter, numbers):
    """
    Priority the numbers.
    """
    priority_numbers = []
    most_numbers = retrieve_most_number(counter)
    for v in most_numbers:
        key = v[0]
        values = sorted(v[1], key=lambda x: int(x))
        for i in values:
            if str(i) in numbers:
                priority_numbers.append(f'p{key}')
    return priority_numbers

def retrieve_all_priority_numbers(counter, numbers):
    """
    All priority the numbers.
    """
    priority_numbers = []
    for v in numbers:
        priority_numbers.append(retrieve_priority_numbers(counter, v))
    return priority_numbers

priority_numbers = retrieve_all_priority_numbers(counter, numbers_data)
priority_counter = counter_numbers(priority_numbers)
priority_counter = sorted(priority_counter.items(), key=lambda x: x[1], reverse=True)
for c in priority_numbers[2:3]:
    c.sort()
    print(c)
print(priority_counter)

['p2', 'p2', 'p3', 'p4', 'p4', 'p5']
[('p5', 55), ('p3', 36), ('p4', 24), ('p2', 22), ('p6', 18), ('p7', 14), ('p8', 8), ('p1', 3)]


In [211]:
import itertools as it

def combination_numbers(numbers, num_of_combo):
    """
    Combination the numbers.
    """
    combinations = []
    for i in numbers:
        combinations.append(list(it.combinations(i, num_of_combo)))
    return combinations

def all_combination_numbers(numbers, num_of_combo):
    """
    All combination the numbers.
    """
    all_combinations = []
    combinations = combination_numbers(numbers, num_of_combo)
    for i, c in enumerate(combinations):
        # print(i, c)
        for a in c:
            all_combinations.append(a)
    return all_combinations

def counter_combination_numbers(numbers, num_of_combo):
    """
    Count the number of times each combination appears in the last 30 draws.
    """
    combinations = all_combination_numbers(numbers, num_of_combo)
    counter = collections.Counter(combinations)
    return counter

def show_counter_combination_numbers(numbers, num_of_combo):
    """
    Show the number of times each combination appears in the last 30 draws.
    """
    counter = counter_combination_numbers(numbers, num_of_combo)
    counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    for k in counter:
        print(k[1], k[0])

def show_numbers_with_combination(numbers, num_of_combo):
    """
    Show the numbers with combination.
    """
    combinations = combination_numbers(numbers, num_of_combo)
    for index, value in enumerate(numbers):
        print("index", index, "value", value)
        for i, c in enumerate(combinations):
            # print(i, c)
            # skip the combination same of slot with numbers
            if index == i: continue
            for x in c:
                # print(x)
                # result = any(a in x for a in value)
                # if result: print(i, x)
                result = [a for a in value if a in x]
                if len(result) == num_of_combo: print(i, x)

show_numbers_with_combination(numbers_data, 2)
# show_counter_combination_numbers(numbers_data, 2)


index 0 value ['1', '13', '19', '26', '29', '37']
4 ('26', '37')
8 ('1', '29')
18 ('19', '37')
26 ('26', '29')
index 1 value ['2', '15', '20', '30', '47', '48']
5 ('47', '48')
7 ('2', '48')
14 ('2', '15')
20 ('30', '47')
21 ('20', '30')
22 ('20', '48')
26 ('15', '20')
index 2 value ['9', '17', '24', '27', '33', '39']
4 ('17', '24')
index 3 value ['7', '16', '21', '30', '34', '38']
5 ('16', '21')
10 ('30', '38')
12 ('30', '34')
16 ('7', '16')
17 ('30', '34')
20 ('21', '30')
25 ('21', '38')
index 4 value ['11', '17', '24', '26', '37', '41']
0 ('26', '37')
2 ('17', '24')
18 ('11', '37')
21 ('11', '26')
index 5 value ['9', '16', '21', '35', '47', '48']
1 ('47', '48')
3 ('16', '21')
7 ('35', '48')
16 ('16', '48')
20 ('21', '47')
25 ('21', '35')
index 6 value ['1', '27', '32', '35', '38', '44']
10 ('27', '38')
25 ('35', '38')
index 7 value ['2', '13', '35', '39', '43', '48']
1 ('2', '48')
5 ('35', '48')
13 ('39', '48')
22 ('43', '48')
29 ('39', '48')
index 8 value ['1', '17', '25', '29', '42